<a href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Labo-App_Mobile/IdEcorces-ResConv-ConversionVersTFLite-Colab.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter) 
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables.


SVP, déployez toutes les cellules en sélectionnant l'item « Développer les rubriques » de l'onglet « Affichage ».

#  Conversion vers un modèle TensorFlow Lite

Comme son nom l'indique, le convertisseur TensorFlow Lite, <code>tf.lite.TFLiteConverter</code>, convertit un modèle TensorFlow Keras en un modèle TensorFlow Lite plus compact. Identifié par l'extension de fichier .tflite, le format TensorFlow Lite.

Pour les modèles Keras, la conversion se fait en deux étapes:

1.   Conversion Keras à TFlite
2.   Ajout des métadonnées

### Inspiration et droits d'auteur

Ce laboratoire s'inspire de plusieurs oeuvres en logiciels libres qui ont été transformées dont:

<a href="https://www.tensorflow.org/lite/convert" target='_blank'>TensorFlow Lite converter</a> - site Google / Guide TensorFlow pour mobile et objet connecté 

<a href="https://www.tensorflow.org/lite/convert/metadata_writer_tutorial" target='_blank'>TensorFlow Lite Metadata Writer API</a> - site Google / Guide TensorFlow pour mobile et objet connecté 

<a href="https://medium.com/axinc-ai/convert-keras-models-to-tensorflow-lite-e654994fb93c">Convert Keras models to TensorFlow Lite</a> - David Cochard - 2 juin 2021

##### Copyright (c) 2019-2022, The TensorFlow Authors.
##### Copyright (c) 2021, David Cochard
##### Copyright (c) 2022, Claude Coulombe

Le contenu de cette page est sous licence <a href="https://creativecommons.org/licenses/by/4.0/deed.fr" target='_blank'>Creative Commons Attribution 4.0 (CC BY 4.0)</a>,<br/>et les exemples de code sont sous <a href="https://www.apache.org/licenses/LICENSE-2.0" target='_blank'>licence Apache 2.0</a>.

#### Données

Les données sur les écorces d'arbres proviennent de <a href="https://data.mendeley.com/research-data/?search=barknet">BarkNet</a>, une banque en données ouvertes sous licence MIT de 23 000 photos d'écorces d'arbres en haute résolution prises avec des téléphones intelligents par une équipe d'étudiants et de chercheurs du <a href="https://www.sbf.ulaval.ca/" target='_blank'>Département des sciences du bois et de la forêt de l'Université Laval</a> à Québec.</p>

## Installation des bibliothèques Python

### ***Attention!*** Pour convertir un modèle Keras vers TFlite, vous
### devez installer une version plus ancienne de TensorFlow, soit la
### version 2.6.0 à cause d'une incompatibilité avec Flatbuffer

<!-- https://github.com/tensorflow/tensorflow/issues/51590 -->

1. Colab vous demandera une confirmation
<img src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Retrograder_version_TensorFlow.png">

2. Vous devrez redémarrer l'environnement d'exécution. 

Soit en cliquant le bouton « RESTART RUNTIME » qui apparaîtra au bas de la trace d'éxécution de la cellule 

<img src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Redemarrer_environnement_dexecution.png">

ou via le menu « Exécution » et l'item « Redémarrer l'environnement d'exécution ».




In [ ]:
!pip3 uninstall tensorflow
!pip3 install tensorflow==2.6.0
!pip3 install keras==2.6.0

#### Vérifiez que la version de TensoFlow est bien 2.6.0

In [ ]:
import os

import tensorflow as tf
print("Version de TensorFlow :",tf.__version__)

print("Bibliothèques Python chargées")

## Choix et téléversement du modèle Keras à convertir

### Téléversement et décompression d'un modèle Keras sauvegardé sur votre poste local

1. & 2. Vous allez téléverser le modèle Keras entraîné précédemment

<img src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Colab_Importer_Fichier.png"/>

3. La fenêtre de l'outil de fichiers de votre ordinateur s'ouvre alors. Allez chercher le modèle Keras compressé, modele_....zip que vous avez sauvegardé précédemment sur votre ordinateur local.

**Attention!** Le téléchargement peut prendre plusieurs minutes. Assurez-vous que le fichier est entièrement téléversé et que l'icône d'état du téléversement (un cercle jaune, comme ci-dessous) disparaisse après avoir été complet. 

<img style="margin-left:0px" src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Colab_Importer_Fichier-3.png"/>


4. Création de trois répertoires pour les modèles: 1) modèle Keras, 2) modèle TFlite obtenu par conversion et 3) modèle TFlite enrichi de métadonnées.

In [ ]:
try:
    os.mkdir("/content/modele_keras/")
except OSError:
    pass
try:
    os.mkdir("/content/modele_TFlite/")
except OSError:
    pass
try:
    os.mkdir("/content/modele_TFlite_metadonnees/")
except OSError:
    pass

5. Décompressez le fichier modele_....zip en exécutant la commande ci-dessous:

In [ ]:
!unzip /content/modele_*.zip -d /content/modele_keras/ && rm /content/modele_*.zip

In [ ]:
# Chemin vers modèles 
chemin_vers_modele_keras = "/content/modele_keras/"
# Taille du modèle Keras
taille_totale = 0
for chemin, repertoires, fichiers in os.walk(chemin_vers_modele_keras):
    for fichier in fichiers:
        chemin_fichier = os.path.join(chemin, fichier)
        taille_totale += os.path.getsize(chemin_fichier)
taille_modele_keras = taille_totale
print("Taille du modèle Keras: " + str(taille_modele_keras//1.0E+06) + " Mo")

## Chargement du modèle

In [ ]:
modele_keras = tf.keras.models.load_model(chemin_vers_modele_keras)

print("Architecture du modèle entraîné")
modele_keras.summary()


##  Conversion du modèle

Pour convertir efficacement un modèle Keras à un modèle plus compact TFlite, l'outil de conversion `TFLiteConverter` aura besoin d'échantillonner des données typiques. 

L'examen d'un échantillon représentatif des données est requis pour réaliser une quantification (<i>quantization</i>), en fait une réduction de la précision des calculs avec des entiers à 8 bits (INT8 ou TFLITE_BUILTINS_INT8).

Vous allez donc créer des répertoires en conséquence.

### Création des répertoires pour les données

In [ ]:
try:
    os.mkdir("/content/donnees/")
except OSError:
    pass
try:
    os.mkdir("/content/lab_ecorces/")
except OSError:
    pass


### Chargement des données

Pour des raisons pratiques, principalement de bande passante, les données sont conservées sur le site Kaggle.

#### Utilisation de l'IPA (<i>API</i>) de Kaggle pour l'importation directe du jeu de données BarkNet

1. Commencez par installer la bibliothèque Python `kaggle`

In [ ]:
!pip3 install kaggle

2. Si ce n'est déjà fait, devenez membre de Kaggle avec votre adresse de courriel GMail:<br/>

<img src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-1.png"/>

3. Maintenant, vous devez télécharger votre clé privée pour utiliser l'IPA de Kaggle.

4. Cliquez sur l'onglet « account » de votre profil Kaggle

<img src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-2.png"/>

5. Sur la page « Account » cliquez sur le bouton « Create New API Token ».
    
<img style="margin-left:40px;" src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-3.png"/>

6. Téléchargez votre clé privée « kaggle.json » pour l'IPA Kaggle dans un endroit temporaire sur votre poste de travail.

<img style="margin-left:40px;" src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-5.png"/>

7. Maintenant, transférez (téléversez) votre clé privée « kaggle.json » dans votre environnement Colab.

La fenêtre de l'outil de fichiers de votre ordinateur s'ouvre alors. Allez chercher votre clé privée « kaggle.json » que vous avez sauvegardée sur votre  ordinateur.

<img style="margin-left:40px;" src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Colab_Importer_Fichier.png"/>


8. Créer à la racine un répertoire .kaggle et déplacez votre clé privée « kaggle.json » dans ce répertoire.

In [ ]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle -all

9. Maintenant téléchargez le jeu de données réduit « barknet-mini » de 1.6 Go avec la commande suivante:

In [ ]:
# Attention! Jeu réduit de données 1.5 Go - plus rapide à télécharger et à traiter
!kaggle datasets download claudecoulombe/barknet-mini --unzip -p /content/donnees/
repertoire_entree = "/content/donnees/BarkNet-mini"

### Répartition des données

In [ ]:
# Installation des bibliothèques Python `split-folders` et `tqdm`
!pip3 install split-folders tqdm

In [ ]:
# Répartition des données d'entraînement, de validation et de tests
import splitfolders
import pathlib

#### répertoire des données une fois réparties
repertoire_donnees_reparties = "/content/lab_ecorces"
# => train, val, test

nombre_images = len(list(pathlib.Path(repertoire_entree).glob('*/*.jpg')))
print("Nombre total d'images:",nombre_images)

splitfolders.ratio(repertoire_entree, 
                   output=repertoire_donnees_reparties, 
                   seed=42, 
                   ratio = (0.65, 0.15, 0.20)
                   )

print("\nRépartition des données terminée!")

## Conversion du modèle proprement dite

In [ ]:
TAILLE_LOT = 32
HAUTEUR_IMAGE = 150
LARGEUR_IMAGE = 150
TAILLE_IMAGE = (HAUTEUR_IMAGE, LARGEUR_IMAGE)
REPERTOIRE_VALIDATION = "/content/lab_ecorces/val"
chemin_vers_modele_TFlite = "/content/modele_TFlite/"
nom_modele_TFlite = "modele.tflite"

In [ ]:
# Source: Convert Keras models to TensorFlow Lite - David Cochard - 2 juin 2021
# https://medium.com/axinc-ai/convert-keras-models-to-tensorflow-lite-e654994fb93c

import numpy as np
import cv2
import glob
import sys

repertoire_images = REPERTOIRE_VALIDATION
chemins_des_images = glob.glob(repertoire_images+"/*/*")
if len(chemins_des_images)==0:
    print("Images absentes")
    sys.exit(1)
else:
    print("Images présentes")

# création d'un jeu de données de validation
donnees_validation =[]
for fichier_image in chemins_des_images:
    image = cv2.imread(fichier_image) #BGR
    image = cv2.resize(image,(150, 150))
    tableau_image = np.asarray(image, dtype=np.float32)
    tableau_image = np.expand_dims(tableau_image, axis=0)
    tableau_image = tableau_image/255.0
    donnees_validation.append(tableau_image)

def generateur_donnees_representatives():
    for i in range(len(donnees_validation)):
        yield [donnees_validation[i]]

print("Code de génération prêt")

In [ ]:
# Conversion et quantification
convertisseur = tf.lite.TFLiteConverter.from_keras_model(modele_keras) 
convertisseur.optimizations = [tf.lite.Optimize.DEFAULT]
convertisseur.representative_dataset = generateur_donnees_representatives
convertisseur.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
convertisseur.inference_input_type = tf.uint8
convertisseur.inference_output_type = tf.uint8
modele_tflite = convertisseur.convert()

# Sauvegarde du modèle TFlite
with open(chemin_vers_modele_TFlite+nom_modele_TFlite, 'wb') as fichier:
    fichier.write(modele_tflite)
    
print("Conversion terminée!")

In [ ]:
# Taille du modèle TFlite
taille_modele_TFlite = os.path.getsize(chemin_vers_modele_TFlite+nom_modele_TFlite)
print("La taille du fichier compact TFlite: ",str(taille_modele_TFlite//1.0E+06) + " Mo")

## Ajout des métadonnées

### Vous aurez besoin de bibliothèques de support de TFlite

In [ ]:
!pip3 install tflite-support-nightly

In [ ]:
etiquettes = [
    'BOJ','BOP','CHR','EPB','EPN',
    'EPO','EPR','ERR','ERS','FRA',
    'HEG','MEL' ,'ORA','OSV','PET',
    'PIB','PIR','PRU','SAB','THO']
print("Nombre d'étiquetttes:", len(etiquettes))


with open('etiquettes.txt', 'w') as fichier_etiquettes:
    for etiquette in etiquettes:
        fichier_etiquettes.write(etiquette + "\n")

In [ ]:
# https://www.tensorflow.org/lite/convert/metadata_writer_tutorial
from tflite_support.metadata_writers import image_classifier
from tflite_support.metadata_writers import writer_utils

ImageClassifierWriter = image_classifier.MetadataWriter
# Des paramètres de normalisation sont requis lors du traitement des images. 
# https://www.tensorflow.org/lite/convert/metadata#normalization_and_quantization_parameters)
_ENTREE_MOYENNE_NORMALISEE = 127.5
_ENTREE_ECART_TYPE_NORMALISE = 127.5
_CHEMIN_FICHIER_MODELE_TFLITE = "/content/modele_TFlite/modele.tflite"
_CHEMIN_FICHIER_ETIQUETTES = "/content/etiquettes.txt"
_CHEMIN_FICHIER_MODELE_TFLITE_METADONNEES = "/content/modele_TFlite_metadonnees/modele_Keras_vers_TFlite_metadonnees.tflite"

# Créer le générateur de métadonnées
generateur_metadonnees = ImageClassifierWriter.create_for_inference(
    writer_utils.load_file(_CHEMIN_FICHIER_MODELE_TFLITE), 
    [_ENTREE_MOYENNE_NORMALISEE], [_ENTREE_ECART_TYPE_NORMALISE],
    [_CHEMIN_FICHIER_ETIQUETTES])

# Vérifier les métadonnées créées par le générateur
print(generateur_metadonnees.get_metadata_json())

# Intégrer les métadonnées dans le modèle TFlite
writer_utils.save_file(generateur_metadonnees.populate(), _CHEMIN_FICHIER_MODELE_TFLITE_METADONNEES)

## Télécharger le modèle TFlite avec métadonnées sur votre poste local

In [ ]:
from google.colab import files

files.download(_CHEMIN_FICHIER_MODELE_TFLITE_METADONNEES)

In [ ]:
print("Fin de l'exécution du carnet IPython")